# 🗺️ Grid Path Search Lab
**SE 444: Introduction to AI - Prof. Anis Koubaa**

## 🎯 Learning Goals
By the end of this lab, you will:
- Understand how to model pathfinding as an AI problem using PEAS
- Compare 4 search algorithms: BFS, DFS, UCS, and A*
- See how heuristics make search more efficient

---


## 🧠 PEAS Framework for Grid Pathfinding

Before coding, let's understand our problem using the **PEAS framework**:

| Component | Grid Pathfinding |
|-----------|------------------|
| **🎯 Performance** | Find shortest path, minimize steps taken |
| **🌍 Environment** | 2D grid with obstacles and free spaces |
| **👁️ Actuators** | Move up, down, left, right |
| **🔍 Sensors** | Current position, can see entire grid |

### Environment Properties:
- **Observable**: Agent can see the entire grid
- **Deterministic**: Same action always produces same result  
- **Static**: Grid doesn't change during search
- **Discrete**: Finite positions and actions
- **Known**: Agent knows the grid layout


## 📍 The Problem: Grid Navigation

**Goal**: Find a path from Start (S) to Goal (G) while avoiding obstacles (█)

### Example Grid:
```
S . . . .
█ █ . █ .  
█ . . . .
█ . █ █ .
. . . . G
```

**Legend**: 
- `S` = Start position (0,0) - top-left corner
- `G` = Goal position (4,4) - bottom-right corner
- `.` = Free space (can move here)
- `█` = Obstacle (cannot pass through)


## 🧠 PEAS Framework
| Component | Grid Pathfinding |
|-----------|------------------|
| **🎯 Performance** | Find shortest path, minimize steps |
| **🌍 Environment** | 2D grid with obstacles and free spaces |
| **👁️ Actuators** | Move up, down, left, right |
| **🔍 Sensors** | Current position, can see entire grid |

**Environment**: Observable, Deterministic, Static, Discrete, Known


## 🔍 Search Algorithms We'll Compare

Now we'll implement 4 different search algorithms to solve this problem:

| Algorithm | Strategy | Guarantee | Best For |
|-----------|----------|-----------|----------|
| **BFS** | Explore level by level | Shortest path | Unweighted graphs |
| **DFS** | Go deep, then backtrack | Finds any path | Memory efficiency |
| **UCS** | Lowest cost first | Optimal cost | Weighted graphs |
| **A*** | Use heuristic guidance | Optimal + efficient | Smart pathfinding |

**Key Questions**: 
- Which will find the 8-step path shown above? 
- Which will explore the fewest nodes? 
- Which will be fastest?

Let's find out by coding! 🚀


---
## 🛠️ Setup
Time to code! First, let's import our libraries and check our environment.


In [40]:
# Import required libraries
from collections import deque
import heapq
import time

# Check if we're running in Google Colab or locally
try:
    import google.colab
    print("📍 Running in Google Colab")
except ImportError:
    print("📍 Running locally")

print("✅ Environment ready! Let's start implementing search algorithms.")
print("🎯 Our goal: Find path from (0,0) to (4,4) with minimum steps!")


📍 Running locally
✅ Environment ready! Let's start implementing search algorithms.
🎯 Our goal: Find path from (0,0) to (4,4) with minimum steps!


In [41]:
# Setup
from collections import deque
import heapq
import time

# Define test grid (0=free, 1=obstacle)
sample_grid = [
    [0, 0, 0, 0, 0],  # S . . . .
    [1, 1, 0, 1, 0],  # █ █ . █ .
    [1, 0, 0, 0, 0],  # █ . . . .
    [1, 0, 1, 1, 0],  # █ . █ █ .
    [0, 0, 0, 0, 0]   # . . . . G
]

start_pos = (0, 0)
goal_pos = (4, 4)

print("✅ Setup complete!")


✅ Setup complete!


In [42]:
# Let's show what a solution looks like!
def show_solution(grid, path, start, goal, title="Solution Path"):
    print(f"🏆 {title}")
    print("=" * 20)
    
    for row in range(len(grid)):
        for col in range(len(grid[0])):
            pos = (row, col)
            if pos == start:
                print("S", end=" ")
            elif pos == goal:
                print("G", end=" ")
            elif grid[row][col] == 1:
                print("█", end=" ")
            elif pos in path:
                print("*", end=" ")  # Path marked with *
            else:
                print(".", end=" ")
        print()
    
    print(f"\n📏 Path length: {len(path) - 1} steps")
    print(f"📍 Path: {path}")

# Example optimal path from (0,0) to (4,4) avoiding obstacles
optimal_path = [(0,0), (0,1), (0,2), (1,2), (2,2), (2,3), (2,4), (3,4), (4,4)]

show_solution(sample_grid, optimal_path, start_pos, goal_pos)


🏆 Solution Path
S * * . . 
█ █ * █ . 
█ . * * * 
█ . █ █ * 
. . . . G 

📏 Path length: 8 steps
📍 Path: [(0, 0), (0, 1), (0, 2), (1, 2), (2, 2), (2, 3), (2, 4), (3, 4), (4, 4)]


## 🏗️ GridSearcher Class
We'll create a helper class that contains all our search algorithms and utility functions.

**What you'll see in the code:**
- `__init__()`: Set up the grid and tracking variables
- `get_neighbors()`: Find valid moves from current position  
- `reconstruct_path()`: Build the solution path from start to goal
- `visualize()`: Show the grid with the path

**Your task**: Run the cell below to create our search framework.


## 🏗️ GridSearcher Class
First, let's create our base class with helper functions that all algorithms will use.


In [43]:
class GridSearcher:
    def __init__(self, grid):
        self.grid = grid
        self.rows = len(grid)
        self.cols = len(grid[0]) if grid else 0
        self.nodes_explored = 0
        
    def reset_stats(self):
        """Reset search statistics for fair comparison"""
        self.nodes_explored = 0
    
    def is_valid(self, row, col):
        """Check if position is valid and passable"""
        return (0 <= row < self.rows and 0 <= col < self.cols and self.grid[row][col] == 0)
    
    def get_neighbors(self, pos):
        """Get valid neighboring positions (up, down, left, right)"""
        row, col = pos
        neighbors = []
        for dr, dc in [(-1,0), (1,0), (0,-1), (0,1)]:  # up, down, left, right
            new_row, new_col = row + dr, col + dc
            if self.is_valid(new_row, new_col):
                neighbors.append((new_row, new_col))
        return neighbors
    
    def manhattan_distance(self, pos1, pos2):
        """Calculate Manhattan distance heuristic for A*"""
        return abs(pos1[0] - pos2[0]) + abs(pos1[1] - pos2[1])
    
    def reconstruct_path(self, parent, start, goal):
        """Build path from start to goal using parent pointers"""
        path = []
        current = goal
        while current is not None:
            path.append(current)
            current = parent.get(current)
        path.reverse()
        return path
    
    def visualize(self, path=None):
        """Display grid with optional path"""
        print("\\n📍 Grid: S=Start, G=Goal, █=Obstacle, .=Empty, *=Path")
        for row in range(self.rows):
            for col in range(self.cols):
                pos = (row, col)
                if self.grid[row][col] == 1:
                    print("█", end=" ")
                elif path and pos == path[0]:
                    print("S", end=" ")
                elif path and pos == path[-1]:
                    print("G", end=" ")
                elif path and pos in path:
                    print("*", end=" ")
                else:
                    print(".", end=" ")
            print()
        print()

print("✅ GridSearcher class ready with helper functions!")


✅ GridSearcher class ready with helper functions!


## 🔄 BFS - Breadth-First Search

### **BFS – Step by Step**

**Problem** → defines the search space: initial_state, goal_test, and possible actions.

**Start Node** → create from initial state (starting point). If it's already the goal → return.

**Frontier (Queue)** → list of nodes to explore, in FIFO order (first in, first out).

**Reached (Set)** → keeps track of visited states to avoid repeats.

**Loop** → while frontier not empty:
1. Pop the first node from frontier
2. For each action from this state → create a child node (new state)
3. If child state not in Reached:
   - If goal → return child (solution found)
   - Else add child to Frontier and mark in Reached

**End** → if frontier empty → return failure (no solution).

### **🎯 What to observe:**
- **Explores level by level** (all depth 1, then all depth 2, etc.)
- **Guarantees shortest path** in unweighted graphs
- **Uses more memory** but finds optimal solution


In [44]:
def bfs(self, start, goal):
    """BFS: Explore level by level using queue (FIFO)"""
    # 📊 Reset statistics and start timer
    self.reset_stats()
    start_time = time.time()
    
    # 🏁 Start Node: Initialize with starting position
    queue = deque([start])          # Frontier: FIFO queue
    visited = {start}               # Reached: Set of visited states
    parent = {start: None}          # Track parents for path reconstruction
    
    # 🔄 Main Loop: While frontier not empty
    while queue:
        current = queue.popleft()   # Pop first node (FIFO)
        self.nodes_explored += 1
        
        # 🎯 Goal Test: Check if we reached the goal
        if current == goal:
            path = self.reconstruct_path(parent, start, goal)
            return path, {
                'algorithm': 'BFS',
                'path_length': len(path) - 1,
                'nodes_explored': self.nodes_explored,
                'time_ms': (time.time() - start_time) * 1000
            }
        
        # 🧭 Expand: Get all possible next states
        for neighbor in self.get_neighbors(current):
            if neighbor not in visited:  # If not in Reached set
                visited.add(neighbor)    # Mark as reached
                parent[neighbor] = current  # Remember parent
                queue.append(neighbor)      # Add to frontier
    
    # ❌ Failure: No path found
    return None, {'algorithm': 'BFS', 'path_length': -1, 
                  'nodes_explored': self.nodes_explored, 
                  'time_ms': (time.time() - start_time) * 1000}

# Add BFS method to our class
GridSearcher.bfs = bfs
print("✅ BFS algorithm implemented!")


✅ BFS algorithm implemented!


## ⬇️ DFS - Depth-First Search

### **DFS – Step by Step**

**Problem** → same search space as BFS but different exploration strategy.

**Start Node** → create from initial state. If it's already the goal → return.

**Frontier (Stack)** → list of nodes to explore, in LIFO order (last in, first out).

**Reached (Set)** → keeps track of visited states to avoid infinite loops.

**Loop** → while frontier not empty:
1. Pop the **last** node from frontier (LIFO)
2. For each action from this state → create a child node
3. If child state not in Reached:
   - If goal → return child (solution found) 
   - Else add child to Frontier and mark in Reached

**Depth Limit** → prevent infinite paths by limiting maximum depth.

### **🎯 What to observe:**
- **Goes deep first** then backtracks when stuck
- **Uses less memory** than BFS (only stores current path)
- **May find suboptimal paths** but can be faster in some cases


In [45]:
def dfs(self, start, goal, max_depth=50):
    """DFS: Go deep first, then backtrack using stack (LIFO)"""
    # 📊 Reset statistics and start timer
    self.reset_stats()
    start_time = time.time()
    
    # 🏁 Start Node: Initialize with starting position and depth
    stack = [(start, 0)]            # Frontier: LIFO stack with (position, depth)
    visited = {start}               # Reached: Set of visited states
    parent = {start: None}          # Track parents for path reconstruction
    
    # 🔄 Main Loop: While frontier not empty
    while stack:
        current, depth = stack.pop()  # Pop last node (LIFO)
        self.nodes_explored += 1
        
        # 🎯 Goal Test: Check if we reached the goal
        if current == goal:
            path = self.reconstruct_path(parent, start, goal)
            return path, {
                'algorithm': 'DFS',
                'path_length': len(path) - 1,
                'nodes_explored': self.nodes_explored,
                'time_ms': (time.time() - start_time) * 1000
            }
        
        # 🚫 Depth Limit: Don't go beyond max depth
        if depth < max_depth:
            # 🧭 Expand: Get all possible next states
            for neighbor in reversed(self.get_neighbors(current)):  # Reverse for consistency
                if neighbor not in visited:  # If not in Reached set
                    visited.add(neighbor)    # Mark as reached
                    parent[neighbor] = current  # Remember parent
                    stack.append((neighbor, depth + 1))  # Add to frontier with incremented depth
    
    # ❌ Failure: No path found within depth limit
    return None, {'algorithm': 'DFS', 'path_length': -1, 
                  'nodes_explored': self.nodes_explored, 
                  'time_ms': (time.time() - start_time) * 1000}

# Add DFS method to our class
GridSearcher.dfs = dfs
print("✅ DFS algorithm implemented!")


✅ DFS algorithm implemented!


## 💰 UCS - Uniform-Cost Search

### **UCS – Step by Step**

**Problem** → same as BFS but considers **path cost** not just number of steps.

**Start Node** → create from initial state with cost = 0.

**Frontier (Priority Queue)** → ordered by **path cost** g(n), lowest cost first.

**Reached (Dict)** → tracks visited states with their **best known cost**.

**Loop** → while frontier not empty:
1. Pop node with **lowest cost** from frontier
2. If already processed with better cost → skip
3. Mark as processed
4. For each action → create child with cost = parent_cost + step_cost
5. If child not reached OR new cost < old cost:
   - If goal → return child (optimal solution found)
   - Else update cost and add to frontier

### **🎯 What to observe:**
- **Always expands lowest cost node** first
- **Guarantees optimal solution** when step costs vary
- **Same as BFS** when all step costs equal (like our grid)


In [46]:
def ucs(self, start, goal):
    """UCS: Expand lowest cost node first using priority queue"""
    # 📊 Reset statistics and start timer
    self.reset_stats()
    start_time = time.time()
    
    # 🏁 Start Node: Initialize with cost = 0
    pq = [(0, start)]               # Frontier: Priority queue (cost, position)
    costs = {start: 0}              # Reached: Best known cost to each state
    parent = {start: None}          # Track parents for path reconstruction
    visited = set()                 # Processed states
    
    # 🔄 Main Loop: While frontier not empty
    while pq:
        current_cost, current = heapq.heappop(pq)  # Pop lowest cost node
        
        # ⏭️ Skip if already processed with better cost
        if current in visited:
            continue
        visited.add(current)        # Mark as processed
        self.nodes_explored += 1
        
        # 🎯 Goal Test: Check if we reached the goal
        if current == goal:
            path = self.reconstruct_path(parent, start, goal)
            return path, {
                'algorithm': 'UCS',
                'path_length': len(path) - 1,
                'nodes_explored': self.nodes_explored,
                'time_ms': (time.time() - start_time) * 1000
            }
        
        # 🧭 Expand: Get all possible next states
        for neighbor in self.get_neighbors(current):
            new_cost = current_cost + 1  # Each step costs 1 in our grid
            
            # 💰 Cost Check: Update if better path found
            if neighbor not in costs or new_cost < costs[neighbor]:
                costs[neighbor] = new_cost      # Update best cost
                parent[neighbor] = current      # Remember parent
                heapq.heappush(pq, (new_cost, neighbor))  # Add to frontier
    
    # ❌ Failure: No path found
    return None, {'algorithm': 'UCS', 'path_length': -1, 
                  'nodes_explored': self.nodes_explored, 
                  'time_ms': (time.time() - start_time) * 1000}

# Add UCS method to our class
GridSearcher.ucs = ucs
print("✅ UCS algorithm implemented!")


✅ UCS algorithm implemented!


## ⭐ A* Search - The Smart Algorithm

### **A* – Step by Step**

**Problem** → same as UCS but uses **heuristic** to guide search toward goal.

**Heuristic h(n)** → estimate of cost from node n to goal (Manhattan distance).

**Evaluation Function** → f(n) = g(n) + h(n)
- g(n) = actual cost from start to n
- h(n) = estimated cost from n to goal

**Start Node** → create with g=0, h=heuristic(start,goal), f=g+h.

**Frontier (Priority Queue)** → ordered by **f(n)** value, lowest f first.

**Loop** → while frontier not empty:
1. Pop node with **lowest f(n)** from frontier
2. If already processed → skip
3. Mark as processed  
4. For each action → create child with new g, h, and f values
5. If child not reached OR new g < old g:
   - If goal → return child (optimal solution!)
   - Else update costs and add to frontier

### **🎯 What to observe:**
- **Combines actual cost + heuristic estimate**
- **Explores fewer nodes** than BFS/UCS by heading toward goal
- **Still guarantees optimal solution** (if heuristic is admissible)


In [47]:
def astar(self, start, goal):
    """A*: Smart search using f(n) = g(n) + h(n)"""
    # 📊 Reset statistics and start timer
    self.reset_stats()
    start_time = time.time()
    
    # 🏁 Start Node: Initialize with g=0, h=heuristic, f=g+h
    h_start = self.manhattan_distance(start, goal)     # h(start) = heuristic estimate
    pq = [(h_start, 0, start)]      # Frontier: Priority queue (f_cost, g_cost, position)
    g_costs = {start: 0}            # g(n): actual cost from start
    parent = {start: None}          # Track parents for path reconstruction
    visited = set()                 # Processed states
    
    # 🔄 Main Loop: While frontier not empty
    while pq:
        f_cost, g_cost, current = heapq.heappop(pq)  # Pop node with lowest f(n)
        
        # ⏭️ Skip if already processed
        if current in visited:
            continue
        visited.add(current)        # Mark as processed
        self.nodes_explored += 1
        
        # 🎯 Goal Test: Check if we reached the goal
        if current == goal:
            path = self.reconstruct_path(parent, start, goal)
            return path, {
                'algorithm': 'A*',
                'path_length': len(path) - 1,
                'nodes_explored': self.nodes_explored,
                'time_ms': (time.time() - start_time) * 1000
            }
        
        # 🧭 Expand: Get all possible next states
        for neighbor in self.get_neighbors(current):
            tentative_g = g_cost + 1    # g(neighbor) = g(current) + step_cost
            
            # 💰 Cost Check: Update if better path found
            if neighbor not in g_costs or tentative_g < g_costs[neighbor]:
                g_costs[neighbor] = tentative_g             # Update g(n)
                parent[neighbor] = current                  # Remember parent
                h_cost = self.manhattan_distance(neighbor, goal)  # h(n) = heuristic
                f_cost = tentative_g + h_cost              # f(n) = g(n) + h(n)
                heapq.heappush(pq, (f_cost, tentative_g, neighbor))  # Add to frontier
    
    # ❌ Failure: No path found
    return None, {'algorithm': 'A*', 'path_length': -1, 
                  'nodes_explored': self.nodes_explored, 
                  'time_ms': (time.time() - start_time) * 1000}

# Add A* method to our class
GridSearcher.astar = astar
print("✅ A* algorithm implemented!")
print("🧠 Manhattan Distance Heuristic: |x1-x2| + |y1-y2|")


✅ A* algorithm implemented!
🧠 Manhattan Distance Heuristic: |x1-x2| + |y1-y2|


## 🧪 Algorithm Comparison
Now let's test all algorithms and see how they perform! Pay attention to:
- **Path Length**: Which finds shortest path?
- **Nodes Explored**: Which is most efficient?  
- **Time**: Which runs fastest?


In [48]:
# Test all algorithms on our sample grid
def run_comparison(grid_name, grid, start, goal):
    """Run all algorithms and compare results"""
    print(f"\\n{'='*50}")
    print(f"🎯 Testing: {grid_name} | Start: {start} → Goal: {goal}")
    print(f"{'='*50}")
    
    searcher = GridSearcher(grid)
    searcher.visualize()
    
    algorithms = ['bfs', 'dfs', 'ucs', 'astar']
    results = []
    
    # Run each algorithm
    for algo in algorithms:
        try:
            path, stats = getattr(searcher, algo)(start, goal)
            results.append((algo.upper(), path, stats))
        except Exception as e:
            print(f"❌ {algo.upper()} failed: {e}")
    
    # Display results table
    print(f"{'Algorithm':<8} {'Length':<8} {'Nodes':<8} {'Time(ms)':<10} {'Found'}")
    print("-" * 50)
    
    for algo, path, stats in results:
        found = "✅" if path else "❌"
        print(f"{algo:<8} {stats['path_length']:<8} {stats['nodes_explored']:<8} {stats['time_ms']:<10.2f} {found}")
    
    # Show best path visualization
    for algo, path, stats in results:
        if path:
            print(f"\\n🏆 Solution by {algo}:")
            searcher.visualize(path)
            break

# Run the comparison
run_comparison("Sample Grid", sample_grid, start_pos, goal_pos)


\n==================================================
🎯 Testing: Sample Grid | Start: (0, 0) → Goal: (4, 4)
\n📍 Grid: S=Start, G=Goal, █=Obstacle, .=Empty, *=Path
. . . . . 
█ █ . █ . 
█ . . . . 
█ . █ █ . 
. . . . . 

Algorithm Length   Nodes    Time(ms)   Found
--------------------------------------------------
BFS      8        17       0.03       ✅
DFS      10       12       0.01       ✅
UCS      8        17       0.02       ✅
ASTAR    8        12       0.02       ✅
\n🏆 Solution by BFS:
\n📍 Grid: S=Start, G=Goal, █=Obstacle, .=Empty, *=Path
S * * . . 
█ █ * █ . 
█ . * * * 
█ . █ █ * 
. . . . G 



## 🎯 Your Turn: Experiment!
Create your own grid and test the algorithms. Try different scenarios:
- **Easy grids**: Few or no obstacles
- **Maze grids**: Many obstacles forming corridors  
- **Different start/goal positions**: See how distance affects performance


In [52]:
# 🎨 Create your own test grid (0=free, 1=obstacle)
custom_grid = [
    [0, 0, 1, 0, 0],
    [0, 1, 1, 1, 0],
    [0, 0, 0, 0, 1],
    [1, 1, 0, 1, 0],
    [0, 1, 0, 0, 0]
]

# 📍 Set your own start and goal positions
custom_start = (0, 0)  # Top-left
custom_goal = (4, 4)   # Bottom-right

# 🧪 Test your custom grid!
run_comparison("Your Custom Grid", custom_grid, custom_start, custom_goal)


\n==================================================
🎯 Testing: Your Custom Grid | Start: (0, 0) → Goal: (4, 4)
\n📍 Grid: S=Start, G=Goal, █=Obstacle, .=Empty, *=Path
. . █ . . 
. █ █ █ . 
. . . . █ 
█ █ . █ . 
. █ . . . 

Algorithm Length   Nodes    Time(ms)   Found
--------------------------------------------------
BFS      8        11       0.03       ✅
DFS      8        9        0.02       ✅
UCS      8        11       0.02       ✅
ASTAR    8        11       0.02       ✅
\n🏆 Solution by BFS:
\n📍 Grid: S=Start, G=Goal, █=Obstacle, .=Empty, *=Path
S . █ . . 
* █ █ █ . 
* * * . █ 
█ █ * █ . 
. █ * * G 



## 🎓 Key Takeaways

After running all algorithms, you should observe:

### **Algorithm Characteristics:**
- **BFS**: ✅ Optimal path, 🔴 High memory usage
- **DFS**: 🟡 May find suboptimal path, ✅ Low memory 
- **UCS**: ✅ Optimal path, 🟡 Similar to BFS for equal costs
- **A***: ✅ Optimal path, ✅ Fewest nodes explored

### **Why A* is Superior:**
- **Heuristic guidance** helps it explore toward the goal
- **Combines best of both worlds**: optimal like BFS + efficient like greedy
- **Most practical** for real-world pathfinding problems

### **Real Applications:**
- 🗺️ **GPS Navigation**: A* finds shortest routes
- 🎮 **Game AI**: NPCs navigate using A*
- 🤖 **Robotics**: Robots plan paths with A*
- 🌐 **Network Routing**: Packets find optimal paths

**Great job!** You now understand how search algorithms work and why heuristics matter! 🚀
